In [1]:
from tools import *

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.509 seconds.
Prefix dict has been built successfully.
W1230 21:29:56.272248 164987 init.cc:157] AVX is available, Please re-compile on local machine
Paddle enabled successfully......


In [2]:
rawdata_path='./data/for_modeling.xlsx'
with open(rawdata_path, 'r',encoding='ISO-8859-1') as fp:
    sheets=pandas.read_excel(rawdata_path,sheet_name=None,index_col=0)

sheet1=sheets['Sheet1'] # 表格一，好评 2017*14
sheet2=sheets['Sheet2'] # 表格二，差评 2084*14


In [3]:
sheet=sheet1.append(sheet2) # 合并好评差评 4101*14

In [4]:
appearance, function, service = split_dimentions(
    sheet
)

In [5]:
stopwords_list=get_stopwords_list('cn_stopwords.txt')

In [6]:
appearance_x_y=generate_train_data(appearance,stopwords_list)
function_x_y=generate_train_data(function,stopwords_list)
service_x_y=generate_train_data(service,stopwords_list)


In [7]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
test_size=0.1
appearance_train_x,appearance_test_x,appearance_train_y,appearance_test_y=train_test_split(*appearance_x_y,test_size=test_size)
function_train_x,function_test_x,function_train_y,function_test_y=train_test_split(*function_x_y,test_size=test_size)
service_train_x,service_test_x,service_train_y,service_test_y=train_test_split(*service_x_y,test_size=test_size)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
appearance_vector1 = CountVectorizer() # 一元词袋
function_vector1 = CountVectorizer() # 一元词袋
service_vector1 = CountVectorizer() # 一元词袋
appearance_vector2 = CountVectorizer(ngram_range=[1,2]) # 二元词袋
function_vector2 = CountVectorizer(ngram_range=[1,2]) # 二元词袋
service_vector2 = CountVectorizer(ngram_range=[1,2]) # 二元词袋

In [9]:
appearance_vector=appearance_vector2
function_vector=function_vector2
service_vector=service_vector2


In [10]:
# 标准化(获得 稀疏句-词关系矩阵)
appearance_train_n_x=appearance_vector.fit_transform(appearance_train_x)
function_train_n_x=function_vector.fit_transform(function_train_x)
service_train_n_x=service_vector.fit_transform(service_train_x)

appearance_test_n_x=appearance_vector.transform(appearance_test_x)
function_test_n_x=function_vector.transform(function_test_x)
service_test_n_x=service_vector.transform(service_test_x)


In [41]:
appearance_vector1.fit_transform(appearance_train_x)


<1071x538 sparse matrix of type '<class 'numpy.int64'>'
	with 4063 stored elements in Compressed Sparse Row format>

In [46]:
vector_sample = appearance_vector1.transform([seg_sentence("这鸟真漂亮",stopwords_list)])

In [62]:
vector_sample.indices

array([345], dtype=int32)

In [64]:
appearance_vector1.inverse_transform(vector_sample)

[array(['漂亮'], dtype='<U5')]

In [11]:
# 模型训练
from sklearn.naive_bayes import MultinomialNB
appearance_classifier=MultinomialNB(alpha=0.1).fit(appearance_train_n_x,appearance_train_y)
function_classifier=MultinomialNB(alpha=0.01).fit(function_train_n_x,function_train_y)
service_classifier=MultinomialNB(alpha=0.01).fit(service_train_n_x,service_train_y)


In [12]:
appearance_train_pre=appearance_classifier.predict(appearance_train_n_x)
function_train_pre=function_classifier.predict(function_train_n_x)
service_train_pre=service_classifier.predict(service_train_n_x)

In [40]:
appearance_classifier.predict(
    appearance_vector2.transform([seg_sentence(x,stopwords_list)for x in ["这鸟真漂亮", "不丑", "难看"]])
    )

array([ 1.,  1., -1.])

In [13]:
from sklearn import metrics

In [14]:
print('appearance 训练集准确率：',metrics.f1_score(appearance_train_pre,appearance_train_y,average='micro'))
print('function 训练集准确率：',metrics.f1_score(function_train_pre,function_train_y,average='micro'))
print('service 训练集准确率：',metrics.f1_score(service_train_pre,service_train_y,average='micro'))

appearance 训练集准确率： 0.9710550887021475
function 训练集准确率： 0.9725
service 训练集准确率： 0.9744552967693464


In [15]:
appearance_test_pre=appearance_classifier.predict(appearance_test_n_x)
function_test_pre=function_classifier.predict(function_test_n_x)
service_test_pre=service_classifier.predict(service_test_n_x)

In [17]:
import joblib
joblib.dump(appearance_classifier,'module/naive_bayes/multinomialNB/appearance_classifier.full')
joblib.dump(function_classifier,'module/naive_bayes/multinomialNB/function_classifier.full')
joblib.dump(service_classifier,'module/naive_bayes/multinomialNB/service_classifier.full')

['module/naive_bayes/multinomialNB/service_classifier.full']

In [29]:

joblib.dump(appearance_vector2,'module/naive_bayes/multinomialNB/appearance_vector2.full')
joblib.dump(function_vector2,'module/naive_bayes/multinomialNB/function_vector2.full')
joblib.dump(service_vector2,'module/naive_bayes/multinomialNB/service_vector2.full')

['module/naive_bayes/multinomialNB/service_vector2.full']

In [18]:
print('appearance 测试集准确率：',metrics.f1_score(appearance_test_pre,appearance_test_y,average='micro'))
print('function 测试集准确率：',metrics.f1_score(function_test_pre,function_test_y,average='micro'))
print('service 测试集准确率：',metrics.f1_score(service_test_pre,service_test_y,average='micro'))

appearance 测试集准确率： 0.9833333333333333
function 测试集准确率： 0.8910256410256411
service 测试集准确率： 0.9121621621621622


In [19]:
appearance_test_n_x.shape,function_test_n_x.shape,service_test_n_x.shape,

((120, 2125), (312, 15553), (148, 13063))

In [20]:
appearance_train_n_x.shape,function_train_n_x.shape,service_train_n_x.shape,

((1071, 2125), (2800, 15553), (1331, 13063))